In [ ]:
##Python venv: search_agent_poc

In [1]:
##This module will take the serialized dictionary got out of PDF Parsing ann try to extract
##Semantic Knowldge like identifying 
## 1.Important Objects/Entities
## 2.Deduplicate Entities
## 3.Extracting Relations
## 4.Extract the main Ideas/Topics around Each Page
## 5.Link the different topics via diffrent entities/Objects
## 6.Break down the document by pages instead of Chunks .
## 7.If a page does not fit a chunk then chunk them extract information and then deduplicate the information across
## the page.

#Next Steps:
## 5.Try to Seggregate the BigPDF on Sections.
## 8.Try To Find Common Objects or ideas that link these sections.
## 9.Try 

In [1]:
import pickle
import pandas as pd
#rag_test_input_path='/home/matrix4284/MY_GEN_AI_PROJECTS/RAG/GraphRAG/graphrag-local-ollama/ragtest/input/'+file_name
import os
# importing shutil module
import shutil

from langchain_community.chat_models import ChatOllama

from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

from sentence_transformers import SentenceTransformer
#embeddings = SentenceTransformer("all-MiniLM-L6-v2")

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import XMLOutputParser

In [3]:
pdf_enrichment_output_dir = '../pdf_enrichment/pdf_enriched_output/'
graph_qa_output_dir = './graph_qa_output_networkx/'
graph_builder_output_dir = '../graph_builder/graph_output_networkx/'
#full_filename=file_name+'_'+str(page_idx)+extension
#full_filename

In [4]:
#LLM Model for Prompt Tuning
model_name="llama3.1"
llm = ChatOllama(base_url="http://192.168.50.100:11434",model=model_name,temperature=0.15)


In [5]:
#### Initial Load data (deserialized data).The input data is the enitity extraction as below:
#with open(pdf_enrichment_output_dir+'pdf_enriched_content_dict_phase1_entity_extraction_478_final.pickle', 'rb') as handle:
#    document_dict_deserialized = pickle.load(handle)

####If error happens start the load with intermediate index
with open(pdf_enrichment_output_dir+'pdf_enriched_content_dict_phase5_extract_highligts_478_final.pickle', 'rb') as handle:
    document_dict_deserialized = pickle.load(handle)
    


In [18]:
doc_len=len(document_dict_deserialized)
doc_len

479

In [19]:
persona="""You are a computer technology expert who has mastery in Kubernetes,Docker\n
            Machine Larning,Generative AI,Natural Language Understanding and Computer Vision.\n"""

In [20]:
template1="""You are a computer technology expert who has mastery in Kubernetes,Docker\n
            Machine Larning,Generative AI,Natural Language Understanding and Computer Vision.\n
            You have to deeply study a user query and extract the main entities or objects from the user's query.\n
            The entities or objects can be anything surrounding \n
            technology in which your expetise lies.By Objects or entities i mean it can be a programming Language,\n
            command,framework,database,process,cache,controller,qeueues,schedulers,errors,exceptions or any \n
            components related to technology. \n
            Never extract page number or any kind of number as entity. \n
            surrounding the technology.Please output in form of list of json where each extracted entity will be \n
            represented by a json along with description and category.\n
            The category name can be like \n
            hardware,software,network,application,database,process,thread,container etc.Mention the category name \n
            in third bracket. \n
            After extracting all the entities along with their description and category format as per rules below: \n
            1.For each entity create a json with 3 keys "entity","description" and "category". \n
            2.Each extracted entity must be enlosed within double quotes. \n
            3.There should not be any unterimnated string for "entity","description" and "category".In case it is \n
            unterminated due to any reason put the ending double quote to it.\n
            3.Each extracted entity must be a valid word,phrase or a complete command.\n
            4.Extracted entity should never be a character,part of a word,alphabet,number or any alphaneumeric.\n
            After all the entities of the page are extracted where each entity is a json then only collate all the entity json \m
            into a list of json.
            5.If any of the extracted entity,description value contain character ':',handle it as a part of entity \n
            or decription ,dont use the character ':' found in entity or description as  a delimiter.\n
            Output the collated list of json where each entity is represented by json. \n
            Do not output anything other than the list of json neither heading/decsription before the list of json\n
            nor any decsription/footer below the json \n
            Here is the user query: \n\n {query} \n\n"""

In [21]:
template2="""You have to deeply study a user query and extract the main entities or objects from the user's query.\n
            The entities or objects can be anything surrounding \n
            technology.By Objects or entities i mean it can be a programming Language,\n
            command,framework,database,process,cache,controller,qeueues,schedulers,errors,exceptions or any \n
            components related to technology. \n
            The entity words or phrases extracted needs to be a part of user query,\n
            you must not add extra entities which are not there in the query text.\n
            Please output in form of list of json where each extracted entity will be \n
            represented by a json along with description.\n
            After extracting all the entities along with their description format as per rules below: \n
            1.For each entity create a json with 2 keys "entity" and "description". \n
            2.Each extracted entity must be enlosed within double quotes. \n
            3.There should not be any unterimnated string for entity and description.
            4.Each extracted entity must be a valid word,phrase or a complete command.\n
            5.Extracted entity should never be a character,part of a word,alphabet,number or any alphaneumeric.\n
            After all the entities of the page are extracted where each entity is a json then only collate all the entity json \n
            into a list of json.\n
            Do not output anything other than the list of json neither heading/decsription before the list of json\n
            nor any decsription/footer below the json \n
            Here is the user query: \n\n {query} \n\n"""

In [22]:
final_template=persona+template2

In [23]:
def extract_entities_from_query(query):
    
    
    
    
    #parser = XMLOutputParser()

# Prompt
    prompt = PromptTemplate(
            template=final_template,
            input_variables=["query"],
            #partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    # Chain
    #Commented out By Kaustav 20/08/2024 as str output parser is giving some unwanted text #####
    chain = prompt | llm | StrOutputParser()
   
    




    #Trying out XML Output parser by Kautsva added 20/08/2024####
    #chain = prompt | llm | parser
    
    

    # Score
    #filtered_docs = []
    
    #print("Page Text")
    #print(page_text)

    output = chain.invoke(
        {
                "query": query,
        }
    )
    
    #print('Relationship Output from LLM"')
    #print(output)
    
    if '[' in output:
        json_output=output.split('[')[1]
        json_output='['+json_output
    else:    
        json_output='['+output
    
    #print('JSON OUTPUT:')
    #print(json_output)
    
    

    json_output=reverse(json_output)
    
    if ']' in json_output:
        json_output=json_output.split(']')[1]
        json_output=reverse(json_output)
        json_output= json_output + ']'
    else:
        json_output=reverse(json_output)
        json_output= json_output + ']'
    
    entities_json=json.loads(json_output)
    
    return entities_json

In [24]:
######This code enrcihes the PDF page by page and while doing so it fails after processing 20 to 50 pages####
######currently we are manually tackling this problem by making the pdf dict list into aglobal variable######
######After the llm fails we take the index of the last page done and save the pdf dict list uptil that ####
#####Then as of now i am manually restarting the by loading the saved pdf dict list as the source pdf and ###
#####Using the last done pdf page index +1 as the starting index.So (last Done page Index + 1) is the ######
###starting index###
####The maximum page has been given as 565 as that is the last page where the main content of the document###
####resides.In case of the book Kubernetes in action it is 565.##############################################
####If this is batch process then the last page can be manually seen and used as the variable else one can ####
#####also go for the pdf enrichment of all the pages of the book#############################################

###This manual thing needs to be automated by either langraph or by using agents framework##################
##Definitions of Individual Enrichment Modules######

# Function to reverse a string
def reverse(string):
    string = string[::-1]
    return string

def entity_collector(page_idx):
    
    #print("Page Number")
    #print(page_idx)
    
    entities_jsonl=document_dict_deserialized[page_idx]['entities']
    
    entity_lst=entity_collector_per_page(entities_jsonl)
    
    
    return entity_lst
    
    
    
def parse_query(query,entity_lst):    
    
    ##Relationship Extraction Enrichment
    if len(entity_lst)>0:
        page_relationship_lst_dict=extract_relationship_per_query(query,entity_lst)

        
        print(page_relationship_lst_dict)
        
def parse_query(query,entity_lst):    
    
    ##Relationship Extraction Enrichment
    if len(entity_lst)>0:
        page_relationship_lst_dict=extract_relationship_per_query(query,entity_lst)

        
        print(page_relationship_lst_dict)
    

In [25]:
#document_dict_deserialized[160]

In [26]:
query="how to test whether a pod is running?"
entities=extract_entities_from_query(query)

In [25]:
entities

[{'entity': 'pod', 'description': 'a container-based instance in Kubernetes'},
 {'entity': 'Kubernetes',
  'description': 'an open-source container orchestration system'},
 {'entity': 'running',
  'description': 'the state of a process or application being executed'}]

In [27]:
# Store data (serialize in a pickle) upto page 102
with open(graph_qa_output_dir+'graph_qa_phase1_entity_extraction.pickle', 'wb') as handle:
    pickle.dump(entities, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [48]:


def entity_collector_per_query(entity_lst):

    entities=[]
    
    for entity in entity_lst:
        #print("Entity:")
        #print(entity)
        
    
        entity_json=json.loads(json.dumps(entity))
        
        
        #print("Entity:")
        #print(type(entity_json))
        
        entity_name=entity_json['entity']
        print(entity_name)
        entities.append(entity_name)
    return list(set(entities))

In [47]:
entity_list=entity_collector_per_query(entities)


Entity:
<class 'dict'>
pod
Entity:
<class 'dict'>
Kubernetes
Entity:
<class 'dict'>
test


In [20]:
entity_list

['pod', 'running', 'Kubernetes']

In [21]:
def extract_relationship_per_query(query,entity_lst):

# Prompt
    prompt = PromptTemplate(
            template="""You are a computer technology expert who has mastery in Kubernetes,Docker\n
            Machine Learning,Generative AI,Natural Language Understanding and Computer Vision.\n
            You have to deeply study an user query/question related to a particular technology and extract all \n
            the closest relations between the list of entities provided as input along with the query.\n
            While extracting relations carry out the following steps as per the below rules for each : \n
            1.You must define each relation by means of a json which must have values for three keys source_entity \n
            ,description,destination_entity.
                a.source_entity should have that entity who is the one performing some action that is the subject. \n
                b.destination_entity should have the entity who is the object on which the source entity \n
                carries out \n
                one more more actions.\n
                c.description of the relation will contain a brief summary about what action was being carried \n
                by the source_entity on the destination_entity.\n
            2.Make Sure the extracted source_entity,destination_entity and description are always \n
            within "" and never within''.\n
            3.There should not be any unterimnated string for "source_entity","destination_entity" and \n
            "description"values.In case it is unterminated due to any reason put the ending double quote to it.\n
            4.Each source_entity and destination_entity must be a valid word,phrase or a complete command.\n
            5.source_entity and destination_entity should never be a character,part of a word,alphabet,number \n
            or any alphaneumeric.\n
            6.description of the relation field must be valid sentence and must never be character,\n
            part of a word,alphabet,number. \n
            6.After all the relations have been extracted collate them into a list of json.
            7.Make sure the json is terminated properly.
            Output should only contain the list of json and no other words or character or sentences. \n
            The out json list must be terminated properly.
            Here is the user query: \n\n {query} \n\n and the entities: \n\n {entities} \n\n""",
            input_variables=["query","entities"],
    )

    # Chain
    chain = prompt | llm | StrOutputParser()

    # Score
    #filtered_docs = []

    output = chain.invoke(
        {
                "query": query,"entities":entity_list
        }
    )
    
    #print('Relationship Output from LLM"')
    #print(output)
    
    if '[' in output:
        json_output=output.split('[')[1]
        json_output='['+json_output
    else:    
        json_output='['+output
    
    #print('JSON OUTPUT:')
    #print(json_output)
    
    

    json_output=reverse(json_output)
    
    if ']' in json_output:
        json_output=json_output.split(']')[1]
        json_output=reverse(json_output)
        json_output= json_output + ']'
    else:
        json_output=reverse(json_output)
        json_output= json_output + ']'
        
    
    
    #page_output_json=json.loads(output)
    #return page_output_json
    return json_output

In [23]:
json_output=extract_relationship_per_query(query,entity_list)

In [24]:
json_output

'[\n  {\n    "source_entity": "kubectl",\n    "description": "check if a pod is running using kubectl command",\n    "destination_entity": "pod"\n  },\n  {\n    "source_entity": "Kubernetes",\n    "description": "use Kubernetes to manage and run pods",\n    "destination_entity": "pod"\n  }\n]'